In [18]:
%load_ext autoreload
%autoreload 2

In [22]:
from pathlib import Path
from datetime import date 

folder =  Path("base")

In [2]:
import sys
sys.path.append(folder.absolute().as_posix())

In [3]:
from preprocess_utils import load_jsonb, save_jsonb 
from datasets import Dataset
from urllib.parse import urlparse


for art in load_jsonb("final.jsonb"):
    url = art["url"]
    parsed = urlparse(url).netloc.split(".")
    server = parsed[-1]
    if server == "cz":
        server = parsed[-2]

    save_jsonb([art], folder / f"{server}.jsonb", "a", show_progress=False)



    



KeyboardInterrupt: 

In [21]:
# Add from cz:403 to aktualne becauose of the same domain

# for art in load_jsonb(folder / "cz:443.jsonb"):
#     save_jsonb([art], folder / "aktualne.jsonb", "a", show_progress=False)

# Remove cz:443
import os
os.remove(folder / "cz:443.jsonb")
os.remove(folder / "ihned.jsonb")


In [136]:
from preprocess_utils import articles_num
print(list(articles_num(folder)))

[('seznamzpravy.jsonb', 77442), ('denik.jsonb', 1636639), ('idnes.jsonb', 530855), ('irozhlas.jsonb', 201681), ('novinky.jsonb', 549661), ('aktualne.jsonb', 165798)]


In [30]:
translate_cat = [
    (["zprávy z domova", "z domova", "domov"], "domácí"),
    (["zprávy ze světa", "svět", "zahraničí", "ze světa"], "zahraniční"),
    (
        ["tipy deníku", "rádce", "rady učitele", "eko rady a tipy"],
        "tipy",
    ),
    (
        [
            "nepoužívat - věda",
            "věda a školy",
            "věda a technologie",
            "věda a technika",
            "věda a vesmír",
            "věda a příroda",
            "věda & vesmír",
            "vesmír",
        ],
        "věda",
    ),
    (
        [
            "technet",
            "tech & trendy",
            "tech",
            "technika",
            "hardware",
            "software",
            "internet",
            "web",
            "internet a pc",
            "aplikace",
            "bonusweb",
            "plné hry",
        ],
        "technologie",
    ),
    (
        [
            "osobní finance",
            "hypotéky a půjčky",
            "banky a spoření",
            "pojištení",
            "investice",
            "daně",
            "penze",
        ],
        "finance",
    ),
    (
        [
            "nová auta",
            "automoto",
            "motorismus",
            "auta",
            "moje auto",
            "auta a motorky",
            "motocykly",
            "autohistorie",
            "ojetá auta",
            "život řidiče",
            "motorky",
        ],
        "auto",
    ),
    (
        ["televize", "film", "tv", "film a tv"],
        "kultura",
    ),
    (["lidé"], "člověk"),
    (
        ["na kole po česku", "cyklorady", "cyklotrasy"],
        "kolo",
    ),
    (["stavba", "bydlení", "koupelna", "kuchyně"], "bydlení"),
    (
        ["cestovánía dovolená", "po česku", "cestujeme"],
        "cestování",
    ),
    (
        [
            "životní styl a společnost",
            "ona",
            "žena",
            "ženy",
            "pro ženy" "styl",
            "životnístyl",
            "nakupování",
            "nákupy",
            "móda",
            "vztahy",
            "vztahy a sex",
            "sex",
        ],
        "životní styl",
    ),
    (["zdraví", "zdraví a fitness", "fit", "krása"], "životní styl"),
    (["jídelníček a recepty", "recepty"], "životní styl"),
    (
        ["nepouzivat - kultura", "tipy na kulturu", "tipyna kulturu", "divadlo"],
        "kultura",
    ),
    (["česká ekonomika", "světová ekonomika"], "ekonomika"),
    (["práce a podnikání", "podniky", "byznys", "aktuálně o eet"], "byznys"),
    (
        ["hokej", "ms v hokeji", "ms hokej", "vancouver - hokej", "nhl", "extraliga"],
        "sport",
    ),
    (
        [
            "kraje",
            "regiony",
            "z okolí",
            "praha",
            "brněnsko",
            "blanensko",
            "Českobudějovicko",
            "chebsko",
            "berounsko",
            "děčínsko",
            "bruntálsko",
            "Boleslavsko",
            "Hradecko",
            "Českolipsko",
            "Havlíčkobrodsko",
            "Břeclavsko",
            "Domažlicko",
            "Chrudimsko",
            "Litoměřicko",
            "Ústecko",
            "Karlovarsko",
            "Benešovsko",
            "Chomutovsko",
            "Liberecko",
            "Frýdecko-místecko",
            "Českokrumlovsko",
            "Hranicko",
            "Žďársko",
            "Olomoucko",
            "Hodonínsko",
            "Prachaticko",
            "Krkonoše",
            "Jindřichohradecko",
            "Jihlavsko",
            "Jablonecko",
            "Ostravsko",
            "Plzeňsko",
            "Jihomoravský kraj",
            "Jičínsko",
            "Rakovnicko",
            "Slovácko",
            "Orlicko",
            "Zlínsko",
            "Nymbursko",
            "Žatecko a lounsko",
            "Prostějovsko",
            "Ústecký kraj",
            "Vyškovsko",
            "Moravskoslezský kraj",
            "Středočeský kraj",
            "Písecko",
            "Strakonicko",
            "Tachovsko",
            "Táborsko",
            "Rokycansko",
            "Klatovsko",
            "Novojičínsko",
            "Kladensko",
            "Znojemsko",
            "Kutnohorsko",
            "Teplicko",
            "Svitavsko",
            "Kroměřížsko",
            "Třebíčsko",
            "Mostecko",
            "Opavsko",
            "Pardubicko",
            "Přerovsko",
            "Šumpersko",
            "Kolínsko",
            "Zlínský kraj",
            "Sokolovsko",
            "Příbramsko",
            "Olomoucký kraj",
            "Královéhradecký kraj",
            "Liberecký kraj",
            "Karvinsko",
            "Valašsko",
            "Pelhřimovsko",
            "Pardubický kraj",
            "Plzeňský kraj",
            "Jihočeský kraj",
            "Rychnovsko",
            "Náchodsko",
            "Kraj vysočina",
            "Mělnicko",
            "Karlovarský kraj",
        ],
        "domácí",
    ),
    (
        [
            "ms ve fotbale",
            "česká liga",
            "liga mistrů",
            "evropská liga",
            "fotbal",
            "euro 2016",
            "euro",
        ],
        "sport",
    ),
    (
        [
            "klasická hudba",
            "hudba",
            "mff kv 2010",
            "mff kv 2011",
            "mff kv 2012",
            "mff kv 2014",
            "mff kv 2015",
            "mff kv 2016",
            "mff kv 2017",
            "mff kv 2018",
            "mff kv 2019",
            "mff kv",
        ],
        "kultura",
    ),
    (["basket", "basketbal"], "sport"),
    (["mobily", "telefony", "mobil"], "technologie"),
    (["názory"], "komentáře"),
    (
        [
            "loh 2020",
            "loh 2016 rio de janeiro",
            "tokio 2021",
            "tokio 2020",
            "loh 2012 londýn",
            "peking 2022",
            "vancouver - olympijské deníky",
        ],
        "sport",
    ),
    (
        [
            "tenis",
            "ostatní sporty",
            "zahraniční ligy",
            "atletika",
            "reprezentace",
            "formule 1",
            "zimní sporty",
            "lyže",
            "tour de france",
            "rallye",
            "biatlon",
        ],
        "sport",
    ),
    (["černá kronika"], "krimi"),
    # Domácí as all other politics are in domácí
    (["volby", "politika"], "domácí"),
    (["koktejl", "kuriozity", "zajímavosti", "rady a zajímavosti"], "koktejl"),
    (["revue", "modelky", "celebrity"], "revue"),
    (["příběhy a rozhovory", "rozhovory"], "rozhovory"),
]


translate_cat = dict(
    [(x, y) for items, res in translate_cat for x, y in zip(items, [res] * len(items))]
)

with open("categories.txt") as f:
    cats = set(f.read().splitlines())


In [34]:
from postprocessing_utils import Gender
authors = {}
with open("authors.txt") as f:
    for line in f:
        author, gender = line.split(",")
        gender = Gender.MAN if gender.strip() == "male" else Gender.WOMAN
        authors[author] = gender
    

In [32]:
with open("headline_filter.txt") as f:
    headline_filters = set(f.read().splitlines())

In [33]:
base_servers = list(folder.iterdir())

In [26]:

from typing import Callable, List
from filtering import create_filter_by_stats, create_filter_by_cz_lang, create_config, create_tokenized_filter, create_filter
from preprocess_utils import load_jsonb, save_jsonb
from postprocessing_utils import postprocess_authors, filter_author, add_server, translate, filter_by_set, postprocess_brief, postprocess_category, postprocess_headline, postprocess_date, postprocess_content  ,as_Article,JSONArticleEncoder, add_cum_gender, add_gender
filtered_folder = Path("article_json_filtered")

In [40]:
# Custom config for each server
from filtering import between
configs = {
    "idnes": {},
    "denik": {},
    "aktualne": {},
    "irozhlas": {},
    "seznamzpravy": {},
    "novinky": {},
}

default = {
    "article_length": between(400, None),
    "avg_word_length": between(4.0, None),
    "num_words_ratio": between(0.11, 0.22),
    "headline_length": between(20, None),
    "brief_length": between(40, None),
    "non_alpha_ratio": between(None, 0.045),
    "date": between(date(2000, 1, 1), date(2022, 8, 31)),
}

In [43]:

from functools import reduce


ratio = 0.75
for f in base_servers:
    filters = [
        create_filter_by_cz_lang(),
        create_filter_by_stats(create_config(configs[f.stem], default)),
        create_filter("headline", headline_filters, lambda head,f: head.startswith(f)),
    ]

    augmentations = [add_server(f.stem),
    translate("category", translate_cat, lower=True),
    postprocess_category, filter_by_set("category", cats, lower=True), postprocess_authors, filter_by_set("author", set(authors.keys()), lower=True),postprocess_brief, postprocess_headline, postprocess_content, postprocess_date, add_gender(authors), add_cum_gender]


    articles = load_jsonb(f)
    articles = filter(lambda x: all(f(x) for f in filters), articles)


    articles = (reduce(lambda x, f: f(x), augmentations, article) for article in articles)
    


    as_articles = (as_Article(article) for article in articles)
    save_jsonb(as_articles, filtered_folder / f"{f.stem}.jsonb", encoder=JSONArticleEncoder, show_progress=False)

         



    

100%|██████████| 165798/165798 [07:04<00:00, 390.86it/s]


In [44]:
non_duplicate_folder = Path("no_duplicates")

In [45]:
from tqdm import tqdm
UNIQ_COLS = ["headline", "brief", "content"]
uniq_dicts = {cat: dict() for cat in UNIQ_COLS}



servers = [f for f in filtered_folder.iterdir()]
all_articles = (article for f in servers for article in load_jsonb(f))

articles = sorted(all_articles, key=lambda x: sum(map(lambda val: val != None, x.values())), reverse=True)
aritcles = sorted(articles, key=lambda x: len(x["content"]), reverse=True)
app_articles = []
for article in tqdm(articles):
    append = True
    for col in UNIQ_COLS:
        retrived = article[col]
        if retrived == None:
            continue

        if retrived in uniq_dicts[col]:
            uniq_dicts[col][retrived] += 1
            append = False
            break

        uniq_dicts[col][retrived] = 0

    if append:
        app_articles.append(article)

save_jsonb(app_articles, non_duplicate_folder / "deduplicated.jsonb", show_progress=True)

100%|██████████| 1628645/1628645 [01:20<00:00, 20183.70it/s]


In [ ]:
sum_by_cat = {col:sum(uniq_dicts[col].values()) for col in UNIQ_COLS}
print(sum_by_cat)

In [3]:
from pathlib import Path
from preprocess_utils import load_jsonb, save_jsonb
non_duplicate_folder = Path("no_duplicates")
output_folder = Path("../final_dataset")

In [7]:
# Create splits 85/7.5/7.5 by date
from datetime import date
import random
servers = [f for f in non_duplicate_folder.iterdir()]

all_articles = (article for f in servers for article in load_jsonb(f))
none_articles = (article for article in all_articles if article["date"] is None)
articles = (article for article in all_articles if article["date"] is not None)
sorted_articles = sorted(articles, key=lambda x: date.fromisoformat(x["date"]))
# Randomly insert none articles
for none_art in none_articles:
    index = random.randint(0, len(sorted_articles))
    sorted_articles.insert(index, none_art)

train = sorted_articles[:int(len(sorted_articles) * 0.85)]
dev = sorted_articles[int(len(sorted_articles) * 0.85):int(len(sorted_articles) * 0.925)]
test = sorted_articles[int(len(sorted_articles) * 0.925):]

save_jsonb(train, output_folder / "train.jsonb", show_progress=True)
save_jsonb(dev, output_folder / "dev.jsonb", show_progress=True)
save_jsonb(test, output_folder / "test.jsonb", show_progress=True)



100%|██████████| 122056/122056 [00:06<00:00, 19594.91it/s]
